In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import datetime

plt.rcParams["figure.figsize"] = (20,10)

In [16]:
data = pd.read_csv('icl_train.csv')

In [3]:
def show_data_by_dates(data,column_name,
                       left = datetime.date(2019, 1, 1),
                       right = datetime.date(2019, 2, 1)):
  group = data[[column_name,'day_date']].groupby(by = ['day_date']).sum()
  x = group.index
  y = group[column_name]
  fig, ax = plt.subplots()
  ax.plot_date(x, y,'b-')
  fig.autofmt_xdate()
  ax.set_xlim([left, right])

In [4]:
def plot_timeseries(df, station, start_date, end_date):
    fig, ax = plt.subplots(figsize=(20,8))
    st_df = df[df.zone == station][['date', 'total']]
    st_df = st_df.loc[(st_df.date >= pd.to_datetime(start_date)) & (st_df.date <= pd.to_datetime(end_date))]
    ax.plot(st_df.date, st_df.total, '.--')
    ax.set_title(station)
    plt.show();

In [17]:
#change data
data['total'] = data.iloc[:, 2:].sum(axis=1)
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].apply(lambda x: x.year)
data['day'] = data['date'].apply(lambda x: x.day)
data['hour'] = data['date'].apply(lambda x: x.hour)
data['day_date'] = data['date'].apply(lambda x: x.date())
data['day_date'] = pd.to_datetime(data['day_date'])
data['day_name'] = data['date'].apply(lambda x: x.day_name())
data['day_year'] = data['date'].apply(lambda x: x.dayofyear)                                  


In [6]:
stations = data['zone'].unique()

In [ ]:
#plot_timeseries(data, stations[0], '20190105', '20190107')

In [18]:
def add_lag(df,column_name,lag_length):
  for lag in range(1,lag_length+1):
    df[column_name+'_'+str(lag)] = df[column_name].shift(lag)
  return df

#amount - количество дней/часов в году..
def encode_time(df, column_name, amount):
  df[column_name+'_s'] = df[column_name].apply(lambda x: np.sin((2*np.pi)/amount*x))
  df[column_name+'_c'] = df[column_name].apply(lambda x: np.cos((2*np.pi)/amount*x))
  return df

def moving_average(df, column_name, moving_length):
  df[column_name+'_ma_'+str(moving_length)] = df[column_name].rolling(window=moving_length).mean().shift(1)
  return df

def gaussian_average(df, column_name, moving_length):
  df[column_name+'_ga_'+str(moving_length)] = df[column_name].rolling(window=5, win_type='gaussian').mean(std=0.5).shift(1)
  return df

def exp_average(df, column_name):
  df[column_name+'_ea_'] = df[column_name].ewm(com=0.5).mean().shift(1)
  return df

def day_off(day):
  if day in ['Sunday','Saturday']:
    return 1
  else:
    return 0

def salary_day(day):
  if ((day>24) or (day<6)):
    return 1
  else:
    return 0

# Preprocessing

In [74]:
column_name = 'Вызов специальной бригады'
station = stations[0]
train_df = data[['date','year','day_year','day','hour','zone','day_name',column_name]].query('zone == @station')

In [75]:
moving_length = [6,12,24,48]
moving_column_names = []

train_df = encode_time(train_df,'hour',24)
train_df = encode_time(train_df,'day',30)
train_df = encode_time(train_df,'day_year',365)

train_df = pd.concat([train_df,pd.get_dummies(train_df['year'])],axis=1)
train_df = pd.concat([train_df,pd.get_dummies(train_df['day_name'])],axis=1)
train_df['day_off'] = train_df['day_name'].apply(day_off)
train_df['salary_day'] = train_df['day'].apply(salary_day)

for i in moving_length:
  train_df = moving_average(train_df, column_name, i)
  moving_column_names.append(column_name+'_ma_'+str(i))
for i in moving_length:
  train_df = gaussian_average(train_df, column_name, i)
  moving_column_names.append(column_name+'_ga_'+str(i))
train_df = exp_average(train_df, column_name)
moving_column_names.append(column_name+'_ea_')
moving_column_names.append(column_name)

In [76]:
#Дифференциал для таргета + average
for diff_feature in moving_column_names:
  train_df[diff_feature+'_diff'] = train_df[diff_feature].diff().shift(1)

#Лаги для таргета + average
for lag_feature in moving_column_names:
  train_df = add_lag(train_df,lag_feature,100)

additional_lag_columns = ['salary_day','day_off']
for lag_feature in additional_lag_columns:
  train_df = add_lag(train_df,lag_feature,3)




In [77]:
train_df = train_df.dropna()
train_df = train_df.drop(columns = ['date','year','day_year','day','hour','zone','day_name'])

In [78]:
#Проверка
dts = []
for column in train_df.columns:
  dts.append(train_df[column].dtype)
pd.Series(dts).value_counts()

float64    1031
uint8         9
int64         3
dtype: int64

# LinReg

In [79]:
import sklearn
from sklearn.model_selection import TimeSeriesSplit

In [80]:
train_df = train_df.reset_index()
train_df = train_df.drop(columns = ['index'])
X = train_df.drop(columns = [column_name])
Y = train_df[column_name]

In [ ]:
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
  X_train, X_test = X.loc[train_index], X.loc[test_index]
  y_train, y_test = Y.loc[train_index], Y.loc[test_index]
  # ТУТ УЧИМ И ТЕСТИМ